# Env

In [3]:
cd ..

/Users/johnvh/Desktop/cos484/tree-of-thought-llm


In [1]:
%pip install tree-of-thoughts-llm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.0/123.0 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 44.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 45.7 MB/s eta 0:00:0000:0100:01
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  Using cached certifi-2023.5.7-py3-none-any.whl (156 kB)
  Using cached tqdm-4.65.0-py3-none-any.whl (77 kB)
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached attrs-23.1.0-py3-none-any.whl (61 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.2/123.2 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 10.4 MB/s eta 0:00:00
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 26.4 MB/s eta 0:00:00
  Attempting uninstall: mpmath
    Found existing installation: mpmath 1.2.1
    Uninstal

In [4]:
import json
from tot.prompts.crosswords import propose_prompt, value_prompt
from tot.models import gpt
from tot.tasks.crosswords import MiniCrosswordsEnv

env = MiniCrosswordsEnv()

# Prompt

In [5]:
def prompt_wrap(obs):
    return propose_prompt.format(input=obs)

print(prompt_wrap(env.reset(0)))
# print('---------')
# print(prompt_wrap(env.step('h2. value')[0]))

Let's play a 5 x 5 mini crossword, where each word should have exactly 5 letters.

Current Board:
_____
_____
_____
_____
_____

Unfilled:
h1. An agendum; something to be done: _____
h2. An engine: _____
h3. Pretentious; flowery: _____
h4. A salon; a hall: _____
h5. To mock; to sneer: _____
v1. To heap: _____
v2. An Indian antelope: _____
v3. To intend; to plan; to devise; a nettle; to guess: _____
v4. A nozzle: _____
v5. Desiccator; more dry: _____

Filled:

Changed:


Given the current status, list all possible answers for unfilled or changed words, and your confidence levels (certain/high/medium/low), using the format "h1. apple (medium)". Use "certain" cautiously and only when you are 100% sure this is the correct word. You can list more then one possible answer for each word.



In [12]:
import re
import copy
from tot.models import gpt

def parse_line(input_str):
    # regular expression pattern to match the input string format
    pattern = r'^([hv][1-5])\. ([a-zA-Z]{5,5}) \((certain|high|medium|low)\).*$'

    # use regex to extract the parts of the input string
    match = re.match(pattern, input_str)

    if match:
        # extract the matched groups
        parts = [match.group(1), match.group(2), match.group(3)]
        return parts
    else:
        return None

confidence_to_value = {'certain': 1, 'high': 0.5, 'medium': 0.2, 'low': 0.1}  # TODO: ad hoc

def parse_response(response):
    # split the response into lines
    lines = response.split('\n')

    # parse each line
    parsed_lines = [parse_line(line) for line in lines]

    # filter out the lines that didn't match the format
    parsed_lines = [(line[0].lower() + '. ' + line[1].lower(), confidence_to_value.get(line[2], 0)) for line in parsed_lines if line is not None]

    return parsed_lines if len(parsed_lines) >= 1 else None


def get_candidates_to_scores(env):
    obs = env.render()
    if obs in env.cache: 
        print('cache hit')
        return env.cache[obs]
    print('call gpt')
    responses = gpt(prompt_wrap(obs), model='gpt-4o-mini', n=8)
    candidates_to_scores = {}
    for response in responses:
        parsed_response = parse_response(response)
        if parsed_response:
            for candidate, score in parsed_response:
                candidates_to_scores[candidate] = candidates_to_scores.get(candidate, 0) + score
        # choose candiate with highest score
    # print(sorted(candidates_to_scores.items(), key=lambda x: x[1], reverse=True))
    env.cache[obs] = candidates_to_scores
    return candidates_to_scores

def propose_score(env, idx):
    obs = env.reset(idx)
    done = False
    infos = []
    while not done:
        responses = gpt(prompt_wrap(obs), model='gpt-4o-mini', n=5)
        candidates_to_scores = {}
        for response in responses:
            parsed_response = parse_response(response)
            if parsed_response:
                for candidate, score in parsed_response:
                    candidates_to_scores[candidate] = candidates_to_scores.get(candidate, 0) + score
        # choose candiate with highest score
        print(sorted(candidates_to_scores.items(), key=lambda x: x[1], reverse=True))
        if len(candidates_to_scores) == 0:
            break
        candidates =  sorted(candidates_to_scores, key=candidates_to_scores.get, reverse=True)
        for candidate in candidates:
            env_ = copy.deepcopy(env)
            env_.step(candidate)
            if not any(_ == 2 for _ in env_.status):
                break
        print(candidate)
        # candidate = input()
        obs, r, done, info = env.step(candidate)
        print(obs)
        print(env.steps, info)
        print('-------------------\n\n\n')
        infos.append(info)
    return infos

# DFS

In [13]:
def dfs(env, actions, infos, time_limit, prune, max_per_state):
    # get candidate thoughts
    candidates_to_scores = get_candidates_to_scores(env)
    if len(candidates_to_scores) == 0: return 0, [], []
    print(sorted(candidates_to_scores.items(), key=lambda x: x[1], reverse=True))

    # back up current state
    board, status, steps = env.board.copy(), env.status.copy(), env.steps

    # try each candidate
    cnt_per_state = 0
    for action in sorted(candidates_to_scores, key=candidates_to_scores.get, reverse=True):
        obs, r, done, info = env.step(action)
        r = info['r_word']
        if len(infos) < time_limit and env.steps < 10 and not any(_ == 2 for _ in env.status):  # not violating any existing constraints
            cnt_per_state += 1
            if cnt_per_state > max_per_state: break
            count = env.prompt_status()       
            actions.append(action)  

            print(len(infos))
            print(actions)
            print(env.render_board())
            print(info)
            print(count)
            if infos:
                best = max(infos, key=lambda x: x['info']['r_word'])
                print('best', best)
            print('--------------')
            print()

            info = {'total_step': len(infos), 'env_step': env.steps, 'actions': actions.copy(), 'info': info, 'count': count}
            infos.append(info)
            if not prune or count['impossible'] < 1:  # only continue if the current status is possible
                dfs(env, actions, infos, time_limit, prune, max_per_state)
            actions.pop()
        env.reset(env.idx, board=board.copy(), status=status.copy(), steps=steps)

In [ ]:
# dfs with pruning
infoss = []
for i in range(0, 25, 5):
    env.reset(i)
    infos = []
    actions = []
    dfs(env, actions, infos, 100, prune=True, max_per_state=3)
    infoss.append(infos)
    with open('logs/crosswords/gpt-4o-mini_infoss_dfs_prune.json', 'w') as fout:
        json.dump(infoss, fout)

cache hit
[('h2. motor', 2.7), ('v5. drier', 2.7), ('h1. tasks', 2.5), ('h5. scoff', 1.9), ('h3. grand', 1.8), ('v4. spout', 1.8), ('h4. salon', 1.0), ('v3. meant', 1.0), ('h4. lobby', 0.7), ('h5. scorn', 0.5), ('v5. dryer', 0.5), ('v1. piles', 0.5), ('v2. nilga', 0.2), ('v2. chiru', 0.2), ('v1. stack', 0.2), ('v3. deign', 0.2), ('v2. black', 0.2), ('v2. chink', 0.1)]
0
['h2. motor']
Current Board:
_____
MOTOR
_____
_____
_____

{'r_letter': 0.2, 'r_word': 0.1, 'r_game': False}
{'sure': 1, 'maybe': 0, 'impossible': 0}
--------------

cache hit
[('h1. tasks', 7), ('v5. drier', 6.5), ('v1. amass', 5.4), ('h4. salon', 4.5), ('h5. scoff', 3.5999999999999996), ('v3. think', 2.1), ('h3. grand', 0.9), ('h4. lobby', 0.2), ('h3. fancy', 0.2), ('h4. foyer', 0.2), ('v1. mound', 0.2), ('h5. scorn', 0.2), ('h3. ornat', 0.2), ('h4. rooms', 0.2), ('v5. raser', 0.1)]
1
['h2. motor', 'h1. tasks']
Current Board:
TASKS
MOTOR
_____
_____
_____

{'r_letter': 0.2, 'r_word': 0.1, 'r_game': False}
{'sure': 3,

In [ ]:
# dfs without pruning
infoss = []
for i in range(0, 100, 5):
    env.reset(i)
    infos = []
    actions = []
    dfs(env, actions, infos, 100, prune=False, max_per_state=3)
    infoss.append(infos)
    with open('logs/crosswords/infoss_dfs_no_prune.json', 'w') as fout:
        json.dump(infoss, fout)